#  思路
1. 构造一个class BasicExp，生成基本的四则运算表达式Exp: 输入参数必选：两个操作数（num1, num2），一个操作符(op)构成; 可选：是否带括号（bracelet）;
2. op若不指定，默认选取范围在四个操作符中['+', '-', '*', '/',] 随机选取。
3. bracelet = True 或者 False，决定生成的Exp是否带括号。
4. 默认num1, num2的生成方法：num1,num2 若未指定，默认选取范围【0，100】，随机生成，可以是整数，或者小数；num2要求总是大于等于num1；
5. 根据 op 的不同，预留重载四种num1,num2的生成方法
6. 一个子函数生成Exp： f"({num1} {op} {num2})" if bracelet else f"{num1} {op} {num2}"
7. 一个子函数调用Exp，使用eval()方法得到计算结果

# 数的规则

## 加法限制
1. 如果不允许负值出现，那么 num1 >= 0， num2 >= 0

## 减法限制
1. 如果不允许负值出现，那么 num1 >= num2, num2 >=0

## 乘法限制
1. 如果不允许负值出现，那么 那么 num1 >= 0， num2 >= 0

## 除法限制
0. num2 != 0
1. 如果不允许小数出现，那么num1 = num2*R, R!=0的整数
2. 如果允许余数形式，那么需要特殊处理--overide the multi functions!
3. 如果不允许负值出现，那么 那么 num1 >= 0， num2 > 0

In [15]:
# 允许除法余数形式
MOD = False

# 难度等级
DIFF = 1

# 加减法的极值
AS_MAX = 100
# 乘除法的极值
MD_MAX_LOW  = 9
MD_MAX_HIGH = 100


if   DIFF == 1: # 三年级上
    AS_MAX = 30
    MD_MAX_LOW  = 3
    MD_MAX_HIGH = 10
    
elif DIFF == 2:  # 三年级下
    AS_MAX = 50
    MD_MAX_LOW  = 5
    MD_MAX_HIGH = 50
    
else:
    pass

In [16]:
import random
debug = False

# 占位符, pdf友好
NULL_2 = '\u0000' * 2
NULL_4 = '\u0000' * 4
NULL_8 = '\u0000' * 8
NULL_12 = '\u0000' * 12

# 占位符, print友好
# NULL_4 = ' ' * 4
# NULL_8 = ' ' * 8

# 几个特殊常数
ZERO = 0
ONE  = 1
TWO  = 2
FIVE = 5
HUND = 100

MIN = ZERO



In [17]:
def generate_number(min_val=MIN, max_val=HUND, integer=True, avoidZero=False):
    '''随机生成一个介于 (min_val, max_val) 之间的整数或者小数'''
    while True:
        rnd = random.randint(min_val, max_val) if integer else random.uniform(min_val, max_val)
        if not (avoidZero and rnd == 0):
            break
            
    return rnd

# MIN_ = 0
# MAX_ = 10

# # 生成一个避免0的整数
# num1 = generate_number(MIN_, MAX_, integer=True, avoidZero=True)
# print(num1)

# # 生成一个避免0的小数
# num2 = generate_number(MIN_, MAX_, integer=False, avoidZero=True)
# print(num2)

# # 生成一个不避免0的整数
# num3 = generate_number(MIN_, MAX_, integer=True, avoidZero=False)
# print(num3)

# # 生成一个不避免0的小数
# num4 = generate_number(MIN_, MAX_, integer=False, avoidZero=False)
# print(num4)

In [18]:
class BasicExp:
    def __init__(self, num1=None, num2=None, op=None, bracelet=False, integer=True, positive=True):
        self.op = op if op is not None else random.choice(['+', '-', '*', '/'])
        self.bracelet = bracelet
        self.integer  = integer
        self.positive = positive
        
        self.num1 = num1 if num1 is not None else generate_number(integer=self.integer)
        self.num2 = num2 if num2 is not None else generate_number(integer=self.integer)

        self.op_str = self.op
        self.exp = None
        self.gen = False # a flag make sure only gen once time

    def generate_addition_numbers(self, positive):
        if positive:
            num1 = generate_number(min_val=ONE, max_val=AS_MAX, integer=self.integer)
            num2 = generate_number(min_val=ONE, max_val=AS_MAX, integer=self.integer)
        else:
            num1 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)
            num2 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)
        return num1, num2

    def generate_subtraction_numbers(self, positive):
        if positive:
            num2 = generate_number(min_val=ZERO, max_val=AS_MAX, integer=self.integer)
            num1 = num2 + generate_number(min_val=ZERO, max_val=AS_MAX, integer=self.integer)
        else:
            num1 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)
            num2 = generate_number(min_val=-1*AS_MAX, max_val=AS_MAX, integer=self.integer)        
        return num1, num2

    def generate_multiplication_numbers(self, positive):
        if positive:
            num1 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer)
            num2 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer)
        else:
            num1 = generate_number(min_val=-1*MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer)
            num2 = generate_number(min_val=-1*MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer)
        return num1, num2

    def generate_division_numbers(self, positive):
        if positive:
            num2 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=self.integer, avoidZero=True)
            if self.integer:
                num1 = num2*generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=True)
            else:
                num1 = generate_number(min_val=MD_MAX_LOW, max_val=MD_MAX_HIGH, integer=False)  
        else:
            num2 = generate_number(min_val=-1*MD_MAX_LOW, max_val=MD_MAX_LOW, integer=self.integer, avoidZero=True)
            if self.integer:
                num1 = num2*generate_number(min_val=-1*MD_MAX_HIGH, max_val=MD_MAX_HIGH, integer=True)
            else:
                num1 = generate_number(min_val=-1*MD_MAX_HIGH, max_val=MD_MAX_HIGH, integer=False)  
                
        # print(f"[deg div]: num1={num1}, num2={num2}")
        return num1, num2

    def generate_exp(self):

        if not self.gen: # make sure only run once time
            if   self.op == '+':
                self.num1, self.num2 = self.generate_addition_numbers(self.positive )
                self.op_str = "\uFF0B"
            elif self.op == '-':
                self.num1, self.num2 = self.generate_subtraction_numbers(self.positive )
                self.op_str = "\uFF0D"
            elif self.op == '*':
                self.num1, self.num2 = self.generate_multiplication_numbers(self.positive )
                self.op_str = "\u00D7"
            elif self.op == '/':
                if MOD:
                    pass
                else:
                    self.num1, self.num2 = self.generate_division_numbers(self.positive )
                self.op_str = "\u00F7"
            
            exp = f"{self.num1}{self.op}{self.num2}"
            self.exp     = f"({exp})" if self.bracelet else f"{exp}"
            
            # exp_str = f"{self.num1:<4}{self.op_str:<4}{self.num2:>4}"
            exp_str = f"{self.num1:<8}{self.op_str:<4}{self.num2:>8}"
            self.exp_str = f"({exp_str})" if self.bracelet else f"{exp_str}"
            
            self.gen     = True
        else:
            pass
            
        return self.exp_str
    
    def evaluate_exp(self):
        return eval(self.exp)

    def generateExpStr(self, ind:int):
        exp_str = self.generate_exp()
        result  = self.evaluate_exp()

        if isinstance(result, int):
            formatted_result = result
        else:            # 如果 result 是 float 且为整数，转换为 int
            formatted_result = int(result) if result.is_integer() else f"{result:.4f}"

        formatStr = f"({ind:<2}) | {exp_str}={NULL_8}| 答案：{formatted_result}"
        if debug: print(formatStr)
        return formatStr


In [19]:
def gen_problem_str(nums:int, bracelet:bool= False, integer:bool=True, positive:bool=True):

    all_out = ''
    for i in range (nums):
        #exp = BasicExp(bracelet=random.randint(0,1), integer=1)
        exp = BasicExp(bracelet=bracelet, integer=integer, positive=positive)
        # print(f"i={i}: Generated Expression: {exp.generate_exp()}")
        # print(f"i={i}: Evaluation Result   : {exp.evaluate_exp()}")
        all_out += exp.generateExpStr(i+1) + '\n'

    print(all_out)
    return all_out

In [20]:
def gen_problem_per_day(nums:int, bias:int=50):
    out=''
    out+=f"Level: {DIFF}, Date:_____， Timing:_____， 总题数_____, 错误数_____\n"
    # out+="---"*21 + "\n"
    out+=gen_problem_str(nums, positive=True)
    out+="\n"*2
    return out

In [21]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdf_name = "multi_page_problem"+"_level"+str(DIFF)+".pdf"

# have to use SimSun, otherwise can not generate pdf correctly
font_name = 'SimSun'
font_path = './SimSun.ttf'

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont(font_name, font_path))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName=font_name, name='Sun', leading=15, fontSize=12, alignment=0))  #leading=spacing # 0=左对齐, 1=居中, 2=右对齐
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        problem_text = gen_problem_per_day(nums)
        lines = problem_text.split('\n')
        for line in lines:
            # 调整字符间距，确保对齐效果--not work
            # formatted_line = line.replace('＋', ' ＋ ').replace('－', ' － ').replace('×', ' × ').replace('÷', ' ÷ ')
            # flowables.append(Paragraph(formatted_line, styles['Sun']))
            flowables.append(Paragraph(line, styles['Sun']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)

# 使用函数生成多页PDF
days = 3  # 
nums = 40  # 
print_problem_pdf(days, nums)

(1 ) | 70      ÷          7=        | 答案：10
(2 ) | 14      ＋         17=        | 答案：31
(3 ) | 10      ＋          4=        | 答案：14
(4 ) | 29      －         14=        | 答案：15
(5 ) | 19      ＋         10=        | 答案：29
(6 ) | 14      ＋          8=        | 答案：22
(7 ) | 6       ×          9=        | 答案：54
(8 ) | 50      －         24=        | 答案：26
(9 ) | 3       ×          8=        | 答案：24
(10) | 16      －         15=        | 答案：1
(11) | 29      －         11=        | 答案：18
(12) | 11      －          1=        | 答案：10
(13) | 35      ÷          7=        | 答案：5
(14) | 42      ÷          7=        | 答案：6
(15) | 35      －          5=        | 答案：30
(16) | 6       ＋         14=        | 答案：20
(17) | 7       ×          4=        | 答案：28
(18) | 4       ＋         15=        | 答案：19
(19) | 7       ×          7=        | 答案：49
(20) | 41      －         16=        | 答案：25
(21) | 11      ＋         20=        | 答案：31
(22) | 21      －         15=        | 答案：6
(23) | 45      ÷          5=        